In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import activations, losses
from datasets import load_dataset
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import os

2024-05-21 10:20:45.186869: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-21 10:20:45.612648: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-21 10:20:45.616107: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 10:20:48.545790: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
print("GPUs available:", physical_devices)

GPUs available: []


In [4]:
path = "../../data/processed"
dataset = load_dataset(path)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'encoded_text'],
        num_rows: 86914
    })
    test: Dataset({
        features: ['text', 'encoded_text'],
        num_rows: 15338
    })
})

In [10]:
print(dataset["train"][0])

{'text': 'Malec skóroval padáčkem jako Poborský. Zase střílí góly až v play off V\xa0základní části hokejové extraligy nic, tam brněnský obránce Tomáš\xa0Malec\xa0góly nedává. Naposled se během dvaapadesátikolového maratonu trefil v\xa0sezoně 2013/14. V\xa0play off je to u něho o něčem jiném, tam se slovenský poctivec s\xa0číslem 71 prosazuje každoročně. Povedlo se mu to i letos. Gólem vteřinu před koncem prvního finále v\xa0Liberci poslal Kometu do vedení na 3:2, navíc to byl gól hodně kuriózní.', 'encoded_text': 13}


In [11]:
train_encoded_text = dataset['train']['encoded_text']
test_encoded_text = dataset['test']['encoded_text']

unique_train_encoded_text = list(set(train_encoded_text))
unique_test_encoded_text = list(set(test_encoded_text))

# Sort the unique values
train_set_sorted = sorted(unique_train_encoded_text)
test_set_sorted = sorted(unique_test_encoded_text)

# Check if both contain the same elements
are_elements_same = (train_set_sorted == test_set_sorted)
print(train_set_sorted)
print(test_set_sorted)
print(are_elements_same)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
True


In [12]:
MODEL_NAME = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
def tokenize_and_get_lengths(examples):
    tokenized_examples = tokenizer(examples['text'], truncation=False, padding=False)
    return {'length': [len(tokens) for tokens in tokenized_examples['input_ids']]}

# Použití funkce na celý dataset
measure = dataset.map(tokenize_and_get_lengths, batched=True, remove_columns=['text'])

# Zjištění maximální délky
max_seq_length = max(measure['train']['length'])
print(f'Max lenght of sequence: {max_seq_length}')


Map:   0%|          | 0/86914 [00:00<?, ? examples/s]

Map:   0%|          | 0/15338 [00:00<?, ? examples/s]

Max lenght of sequence: 337


In [14]:
max_seq_length = 512
encodings = tokenizer(dataset['train']['text'], max_length=max_seq_length,truncation=True, padding=True)

In [15]:
input_ids = tf.constant(encodings['input_ids'])
attention_mask = tf.constant(encodings['attention_mask'])
labels = tf.constant(dataset['train']['encoded_text'])

2024-05-21 08:53:14.675359: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-21 08:53:14.691965: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-05-21 08:53:14.699451: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117160072 exceeds 10% of free system memory.
2024-05-21 08:53:15.254081: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117160072 exceeds 10

In [16]:
print(tf.constant(encodings['input_ids']).shape)
print(tf.constant(encodings['attention_mask']).shape)
print(tf.constant(dataset['train']['encoded_text']).shape)

2024-05-21 08:54:50.346506: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117160072 exceeds 10% of free system memory.
2024-05-21 08:54:50.888516: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117160072 exceeds 10% of free system memory.


(86914, 337)
(86914, 337)
(86914,)


In [17]:
def create_tf_dataset(input_ids, attention_mask, labels):
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        },
        labels
    ))
    return dataset

In [18]:
batch_size = 32
train_dataset = create_tf_dataset(input_ids, attention_mask, labels)
train_dataset = train_dataset.shuffle(10000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [19]:
EPOCHS = 3
count_of_categories =24

model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=count_of_categories)
loss = losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
model.fit(train_dataset, batch_size=batch_size, epochs=EPOCHS)

2024-05-21 08:55:01.433985: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized fro

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
